In [1]:
# Add autoreload to the notebook
%load_ext autoreload
%autoreload 2

In [2]:
import json

import openai
import pandas as pd

import guardrails as gd
# from utils import get_examples

from text2sql import Text2Sql

# from utils import sqllite_db_to_prompt, sqllite_db_to_prompt_tables

In [4]:
# Load spider dataset

spider_df = pd.read_json("/Users/shreyarajpal/Downloads/spider/train_spider.json")

In [5]:
dpt_mgmt = spider_df[spider_df["db_id"] == "department_management"]

In [6]:
dpt_mgmt

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."
5,department_management,SELECT name FROM head WHERE born_state != 'Cal...,"[SELECT, name, FROM, head, WHERE, born_state, ...","[select, name, from, head, where, born_state, ...",What are the names of the heads who are born o...,"[What, are, the, names, of, the, heads, who, a...","{'from': {'table_units': [['table_unit', 1]], ..."
6,department_management,SELECT DISTINCT T1.creation FROM department AS...,"[SELECT, DISTINCT, T1.creation, FROM, departme...","[select, distinct, t1, ., creation, from, depa...",What are the distinct creation years of the de...,"[What, are, the, distinct, creation, years, of...","{'from': {'table_units': [['table_unit', 0], [..."
7,department_management,SELECT born_state FROM head GROUP BY born_stat...,"[SELECT, born_state, FROM, head, GROUP, BY, bo...","[select, born_state, from, head, group, by, bo...",What are the names of the states where at leas...,"[What, are, the, names, of, the, states, where...","{'from': {'table_units': [['table_unit', 1]], ..."
8,department_management,SELECT creation FROM department GROUP BY creat...,"[SELECT, creation, FROM, department, GROUP, BY...","[select, creation, from, department, group, by...",In which year were most departments established?,"[In, which, year, were, most, departments, est...","{'from': {'table_units': [['table_unit', 0]], ..."
9,department_management,"SELECT T1.name , T1.num_employees FROM depart...","[SELECT, T1.name, ,, T1.num_employees, FROM, d...","[select, t1, ., name, ,, t1, ., num_employees,...",Show the name and number of employees for the ...,"[Show, the, name, and, number, of, employees, ...","{'from': {'table_units': [['table_unit', 0], [..."


In [7]:
examples = dpt_mgmt[["question", "query"]].to_dict(orient="records")

In [8]:
schema_path = "/Users/shreyarajpal/Downloads/spider/database/department_management/schema.sql"
conn_str = f"sqlite://"

In [9]:
app = Text2Sql(conn_str, examples=examples)

encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)
encoding  cl100k_base
(1, 1536)


In [10]:
few_shot_examples = app.store.search("What is the name of the department with the highest number of employees?", 2)

In [11]:
prompt = app._examples_to_prompt(few_shot_examples)

In [12]:
print(prompt)

INSTRUCTIONS:
---------
What is the average number of employees of the departments whose rank is between 10 and 15?

QUERY:
---------
{
    "generated_sql": "SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15"
}

=====

INSTRUCTIONS:
---------
Which department has more than 1 head at a time? List the id, name and the number of heads.

QUERY:
---------
{
    "generated_sql": "SELECT T1.department_id ,  T1.name ,  count(*) FROM management AS T2 JOIN department AS T1 ON T1.department_id  =  T2.department_id GROUP BY T1.department_id HAVING count(*)  >  1"
}

=====




In [13]:
app("What is the name of the department with the highest number of employees?")

/Users/shreyarajpal/anaconda3/envs/tiff-env/lib/python3.9/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if isinstance(o, (numpy.bool, numpy.bool_)):


('{\n    "generated_sql": "SELECT name FROM department WHERE num_employees = (SELECT MAX(num_employees) FROM department)"\n}',
 {'generated_sql': 'SELECT name FROM department WHERE num_employees = (SELECT MAX(num_employees) FROM department)'})

In [14]:
app.guard.state.most_recent_call.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ You are a data scientist whose job is to write SQL queries.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <sql name="generated_sql" description="Generate SQL for the given natural language instruction."    │ │
    │ │ format="bug-free-sql"/>                                                                                 │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise.                                 │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Here's schema about the database that you can use to generate the SQL query.                            │ │
    │ │ Try to avoid using joins if the data can be retrieved from the same table.                              │ │
    │ │                                                                                                         │ │
    │ │ []                                                                                                      │ │
    │ │                                                                                                         │ │
    │ │ I will give you a list of examples. Write a SQL query similar to the examples below:                    │ │
    │ │                                            

In [13]:
llm_output = '{"generated_sql": "SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1"}'

In [15]:
from guardrails.llm_providers import openai_wrapper

In [32]:
app.guard.parse(
    llm_output=llm_output,
    llm_api=openai_wrapper,
    engine="text-davinci-003",
    prompt_params={
        "nl_instruction": "What is the name of the department with the highest number of employees?",
        "db_info": str(app.sql_schema),
        "examples": "",
    },
    max_tokens=512,
)

KeyError: '\n  "generated_sql"'

In [23]:
app.guard.state.most_recent_call.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ No prompt                                                                                               │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │ {"generated_sql": "SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1"}                   │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
│   │ │ {                                                                                                       │ │
│   │ │     'generated_sql': ReAsk(                                                                             │ │
│   │ │         incorrect_value='SELECT name FROM departments ORDER BY num_employees DESC LIMIT 1',             │ │
│   │ │         error_message='(sqlite3.OperationalError) no such table: departments\n[SQL: SELECT name FROM    │ │
│   │ │ departments ORDER BY num_employees DESC LIMIT 1]\n(Background on this error at:                         │ │
│   │ │ https://sqlalche.me/e/14/e3q8)',                                                                        │ │
│   │ │         fix_value=None,                                                                                 │ │
│   │ │         path=['generated_sql']                                                                          │ │
│   │ │     )                                                                                                   │ │
│   │ │ }                                                                                                       │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
└── ╭────────────────────────────────────────────────── Step 1 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ You are a data scientist whose job is to write SQL queries.                                             │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <sql name="generated_sql" description="Generate SQL for the given natural language instruction."    │ │
    │ │ format="bug-free-sql"/>                                                                                 │ │
    │ │ </output>                                                                                               │ │
    │ │                                            

In [20]:
from guardrails.prompt import Prompt


reask_prompt = Prompt("""

You are a data scientist whose job is to write SQL queries.

@complete_json_suffix_v2

Here's schema about the database that you can use to generate the SQL query.
Try to avoid using joins if the data can be retrieved from the same table.

{{db_info}}

I will give you a list of examples.

{{examples}}

I want to create a query for the following instruction:

{{nl_instruction}}

I was given the following JSON response for this query, which had problems due to incorrect values.

{previous_response}

Help me correct the incorrect values based on the given error messages.
""")

app.guard.reask_prompt = reask_prompt

In [ ]:
What is the name of the department with the highest number of employees?